![RDD key pair](media/10.spark_run_scripts.png)

### Ejecución de un programa Spark y montaje de clusters

#### Comando `spark-submit`

-   Permite lanzar programas Spark a un cluster

-   Ejemplo:
```sh
$ bin/spark-submit --master yarn --deploy-mode cluster \  
     --py-files otralib.zip,otrofich.py \  
     --num-executors 10 --executor-cores 2 \  
     mi-script.py opciones_del_script
```

#### Opciones de `spark-submit`

-   `master`: cluster manager a usar (opciones: `yarn`, `mesos://host:port`, `spark://host:port`, `local[n]`)

-   `deploy-mode`: dos modos de despliegue

    -   `client`: ejecuta el driver en el nodo local

    -   `cluster`: ejecuta el driver en un nodo del cluster

-   `class`: clase a ejecutar (Java o Scala)

-   `name`: nombre de la aplicación (se muestra en el Spark web)

-   `jars`: ficheros jar a añadir al classpath (Java o Scala)

-   `py-files`: archivos a añadir al PYTHONPATH (`.py`,`.zip`,`.egg`)

-   `files`: ficheros de datos para la aplicación

-   `executor-memory`: memoria total de cada ejecutor

-   `driver-memory`: memoria del proceso driver

Para más opciones: `spark-submit --help`

In [1]:
!pip install pyspark

You are using pip version 9.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# Create apache spark context
from pyspark import SparkContext
sc = SparkContext(master="local", appName="Mi app")

In [13]:
# Stop apache spark context
sc.stop()

# Instalación Manual

Siguiendo estas dos guías podemos configurar de manera manual un pequeño cluster con un nodo maestro y dos esclavos, pero es un proceso muy largo y tedioso:
- https://www.tutorialspoint.com/apache_spark/apache_spark_installation.htm
- https://www.linode.com/docs/databases/hadoop/install-configure-run-spark-on-top-of-hadoop-yarn-cluster/

![Spark docker meme](media/09.meme_docker_spark.jpg)

## Instalación Usando Docker

Con base a lo comentado arriba instalar manualmente spark es un proceso largo y tedioso, por lo cual usaremos nuestro **super poder Docker**.

![Spark Docker](media/08.spark_docker.jpg)

Vamos a usar el repositorio oficial de la comunidad europea de big data.

- https://github.com/big-data-europe/docker-spark

Como prerequisito obviamente es necesario tener instalado docker, y adicionalmente docker-compose.

### Docker

#### Prerequisites

Download && install **docker**
- [For Mac](https://download.docker.com/mac/stable/Docker.dmg)
- [For Windows](https://download.docker.com/win/stable/InstallDocker.msi)
- [For Linux](https://docs.docker.com/engine/getstarted/step_one/#docker-for-linux)

Download && install **docker-compose**
- [Instructions](https://docs.docker.com/compose/install/)

Download && install **docker-machine**
- [Instructions](https://docs.docker.com/machine/install-machine/)

## Correr cluster

Pasos para levantar el cluster:

* Clonar repositorio

```sh
$ git clone https://github.com/big-data-europe/docker-spark.git
```

* Entrar al repositorio

```sh
$ cd docker-spark
```

* Correr servicios

```sh
$ docker-compose up
```

En el archivo `docker-compose.yml` está la configuración del cluster con el nodo master y los esclavos.

```yml
version: '2'
services:
  spark-master:
    image: bde2020/spark-master:2.3.1-hadoop2.7
    container_name: spark-master
    ports:
      - "8090:8080"
      - "7077:7077"
    environment:
      - INIT_DAEMON_STEP=setup_spark
      - "constraint:node==<yourmasternode>"

  spark-worker-1:
    image: bde2020/spark-worker:2.3.1-hadoop2.7
    container_name: spark-worker-1
    depends_on:
      - spark-master
    ports:
      - "8091:8081"
    environment:
      - "SPARK_MASTER=spark://spark-master:7077"
      - "constraint:node==<yourmasternode>"

  spark-worker-2:
    image: bde2020/spark-worker:2.3.1-hadoop2.7
    container_name: spark-worker-2
    depends_on:
      - spark-master
    ports:
      - "8092:8081"
    environment:
      - "SPARK_MASTER=spark://spark-master:7077"
      - "constraint:node==<yourworkernode>"

  spark-worker-3:
    image: bde2020/spark-worker:2.3.1-hadoop2.7
    container_name: spark-worker-3
    depends_on:
      - spark-master
    ports:
      - "8093:8081"
    environment:
      - "SPARK_MASTER=spark://spark-master:7077"
      - "constraint:node==<yourworkernode>"
```

## Comprobar estado del cluster

Usando el comando `docker ps` podemos ver todos los contenedores que estan corriendo en la máquina.

![Dodkcer ps](media/11.docker_ps.png)

Luego se puede entrar a cada uno de los servicios usando el nombre del contenedor o el id:

Comando:

```sh
$ docker exec -it <docker_id/docker_name> bash
```

Ejemplo:

```sh
$ docker exec -it spark-master bash
```

Comprobar spark:

```sh
$  ./spark/bin/spark-submit --help
```








## Spark YARN

![Yarn client mode](media/12.yarn_client_mod.png)
![Yarn cluster mode](media/13.yarn_cluster_mode.png)


Fuente: [Spark-on-YARN: Empower Spark Applications on Hadoop Cluster](https://www.slideshare.net/Hadoop_Summit/sparkonyarn-empower-spark-applications-on-hadoop-cluster)


### Parámetros de configuración

Diversos parámetros ajustables en tiempo de ejecución

-   En el script
```python
from pyspark import SparkConf,SparkContext
conf = SparkConf()
conf.set("spark.app.name", "Mi apli")
conf.set("spark.master", "local[2]") # Cluster manager modo local con 2 hilos
conf.set("spark.ui.port", "3600")    # Puerto del interfaz web de Spark (por defecto: 4040)
sc = SparkContext(conf=conf)
```

-   Mediante flags en el `spark-submit`

```sh
$ bin/spark-submit --master local[2] --name "Mi apli" \  
    --conf spark.ui.port=3600 mi-script.py
```    
    
-   Mediante un fichero de propiedades

```sh
$ cat config.conf
spark.master     local[2] 
spark.app.name   "Mi apli" 
spark.ui.port 3600
$ bin/spark-submit --properties-file config.conf mi-script.py
```

Más info: <http://spark.apache.org/docs/latest/configuration.html#spark-properties>

## Ejemplo

Crear un archivo con el siguiente script comando:

Script para crear archivo:

```sh
cat << EOF > /tmp/miscript.py
from pyspark import SparkConf, SparkContext
from operator import add

def main():
    conf = SparkConf()
    conf.set("spark.app.name", "Mi script Python")

    # Iniciamos el SparkContext
    sc = SparkContext(conf=conf)
    sc.setLogLevel("FATAL")

    rdd = sc.parallelize(range(100000)).cache()
    
    rdd2 = rdd.map(lambda x: (x, 2*x))\
              .map(lambda (x,y): (x-100, y**2))\
              .reduceByKey(lambda x,y: x+y)\
              .values()
               
    r = rdd2.reduce(add)
    
    print("Resultado final = {0}".format(r))
    
    # Finalizamos el SparkContext
    sc.stop()
if __name__ == "__main__":
    main()
EOF
```

Correr script:

```sh
$ ./spark/bin/spark-submit --master local[8] /tmp/miscript.py
```